In [1]:
import pandas as pd

df_articles = pd.read_csv('/Users/portergraham/Downloads/shared_articles.csv')
df_interactions = pd.read_csv('/Users/portergraham/Downloads/users_interactions.csv')

df_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [2]:
df_interactions = df_interactions.drop(columns=['userAgent', 'sessionId'])
df_articles = df_articles.drop(columns=['authorSessionId', 'authorUserAgent', 'url'])
df_interactions.head()

,timestamp,eventType,contentId,personId,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,NaN,NaN


In [3]:
n_interactions = len(df_interactions)
n_articles = len(df_articles['contentId'].unique())
n_users = len(df_interactions['personId'].unique())
        
print(f"Number of ratings: {n_interactions}")
print(f"Number of unique movieId's: {n_articles}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_interactions/n_users, 2)}")
print(f"Average ratings per movie: {round(n_interactions/n_articles, 2)}")
print(f"totla possible ratings: {n_articles * n_users}")

Number of ratings: 72312
Number of unique movieId's: 3057
Number of unique users: 1895
Average ratings per user: 38.16
Average ratings per movie: 23.65
totla possible ratings: 5793015


In [4]:
print('Min:\t\t', df_interactions['contentId'].value_counts().min())
print('Quartile 1:\t', df_interactions['contentId'].value_counts().quantile(.25))
print('Median:\t\t',df_interactions['contentId'].value_counts().quantile(.5))
print('Quartile 3:\t', df_interactions['contentId'].value_counts().quantile(.75))
print('Max:\t\t', df_interactions['contentId'].value_counts().max())

Min:		 1
Quartile 1:	 6.0
Median:		 14.0
Quartile 3:	 30.0
Max:		 433


In [5]:
value_counts = df_interactions['contentId'].value_counts()

keep_list = value_counts[value_counts >= 25]

df_interactions = df_interactions.loc[df_interactions['contentId'].isin(keep_list.index)]

df_interactions.shape

(52196, 6)

In [6]:
df_interactions['eventType'] = df_interactions['eventType'].replace({'VIEW': 1, 'LIKE': 2, 'FOLLOW': 3, 'BOOKMARK': 4, 'COMMENT': 5, 'COMMENT CREATED': 5})
df_interactions['eventType'] = df_interactions['eventType'].astype(int)

df_interactions.head()

/var/folders/hr/ylx18m9150l99gttcdcwjsx00000gn/T/ipykernel_12359/3267439547.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_interactions['eventType'] = df_interactions['eventType'].replace({'VIEW': 1, 'LIKE': 2, 'FOLLOW': 3, 'BOOKMARK': 4, 'COMMENT': 5, 'COMMENT CREATED': 5})


,timestamp,eventType,contentId,personId,userRegion,userCountry
1,1465412560,1,8890720798209849691,-1032019229384696495,NY,US
2,1465416190,1,310515487419366995,-1130272294246983140,NaN,NaN
3,1465413895,3,310515487419366995,344280948527967603,NaN,NaN
5,1465413742,1,310515487419366995,-8763398617720485024,MG,BR
7,1465415066,1,-1492913151930215984,4254153380739593270,SP,BR


In [93]:
import numpy as np
from scipy.sparse import csr_matrix

# Count number of movies and users
U = df_interactions['personId'].nunique()
I = df_interactions['contentId'].nunique()

# Create a dictionary to map the user and item IDs to consecutive integers with no missing values
user_mapper = dict(zip(np.unique(df_interactions['personId']), list(range(U))))
item_mapper = dict(zip(np.unique(df_interactions['contentId']), list(range(I))))

user_inv_mapper = dict(zip(list(range(U)), np.unique(df_interactions['personId'])))
item_inv_mapper = dict(zip(list(range(I)), np.unique(df_interactions['contentId'])))

user_index = [user_mapper[i] for i in df_interactions['personId']]
item_index = [item_mapper[i] for i in df_interactions['contentId']]

X = csr_matrix((df_interactions['eventType'], (user_index, item_index)), shape=(U, I))
print(X)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 27552 stored elements and shape (1829, 928)>
  Coords	Values
  (0, 21)	1
  (0, 50)	1
  (0, 60)	1
  (0, 64)	1
  (0, 100)	8
  (0, 104)	1
  (0, 118)	1
  (0, 127)	1
  (0, 139)	1
  (0, 143)	2
  (0, 144)	2
  (0, 147)	1
  (0, 179)	1
  (0, 223)	2
  (0, 253)	1
  (0, 256)	2
  (0, 273)	1
  (0, 281)	1
  (0, 347)	2
  (0, 421)	1
  (0, 427)	2
  (0, 434)	3
  (0, 500)	1
  (0, 504)	1
  (0, 531)	1
  :	:
  (1828, 454)	13
  (1828, 480)	4
  (1828, 485)	1
  (1828, 491)	3
  (1828, 493)	1
  (1828, 515)	1
  (1828, 534)	1
  (1828, 578)	2
  (1828, 604)	2
  (1828, 611)	1
  (1828, 617)	10
  (1828, 635)	1
  (1828, 652)	19
  (1828, 693)	1
  (1828, 706)	2
  (1828, 728)	1
  (1828, 732)	1
  (1828, 805)	1
  (1828, 830)	5
  (1828, 837)	9
  (1828, 850)	1
  (1828, 864)	2
  (1828, 883)	10
  (1828, 887)	1
  (1828, 909)	1


In [7]:
from surprise import Dataset, Reader

# Define a reader object to specify rating scale (eventType values)
reader = Reader(rating_scale=(1, 5))  # Min and max values in eventType

# Load the data into Surprise
data = Dataset.load_from_df(df_interactions[['personId', 'contentId', 'eventType']], reader)

In [8]:
from surprise.model_selection import train_test_split

# Split into 80% train, 20% test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
from surprise import SVD
from surprise import accuracy

# Initialize the SVD model
model = SVD()

# Train on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model using RMSE (Root Mean Square Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 0.8643
RMSE: 0.8642777025953378


In [10]:
user_id = -8845298781299428018
item_id = -3499919498720038879
predicted_rating = model.predict(user_id, item_id).est
print(f"Predicted rating: {predicted_rating}")


Predicted rating: 1.6569118559733171


In [13]:
# Create a set of known interactions
known_interactions = set(zip(df_interactions['personId'], df_interactions['contentId']))

predictions = []
for user_id in df_interactions['personId'].unique():
    for item_id in df_interactions['contentId'].unique():
        if (user_id, item_id) not in known_interactions:  # Only predict unseen items
            pred = model.predict(user_id, item_id)
            predictions.append([user_id, item_id, pred.est])

df_predictions = pd.DataFrame(predictions, columns=['personId', 'contentId', 'predicted_eventType'])

df_predictions = df_predictions.sort_values(by=['personId', 'predicted_eventType'], ascending=[True, False])
df_predictions = df_predictions.groupby('personId').head(5) 

df_predictions.to_csv('collaborative_filtering.csv', index=False)

print("Filtered recommendations saved to 'recommendations.csv'")

Filtered recommendations saved to 'recommendations.csv'
